In [7]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd

ticker = yf.Ticker("RELIANCE.NS")
hist = ticker.history(period="10mo", interval="1wk")
weekly_close = hist['Close']
print("Weekly Close Data:")
weekly_close = weekly_close.map(lambda x: round(x, 2))
#print(weekly_close)

hist = ticker.history(period="15d", interval="1d")
daily_close = hist['Close']
print("Daily Close Data:")
daily_close = daily_close.map(lambda x: round(x, 2))

macd_data = ta.macd(weekly_close, fast=12, slow=26, signal=9)
RSI = ta.rsi(weekly_close, length=14)
combined_df = pd.concat([weekly_close, macd_data, RSI], axis=1)

print(combined_df)



Weekly Close Data:
Daily Close Data:
                             Close  MACD_12_26_9  MACDh_12_26_9  \
Date                                                              
2024-08-26 00:00:00+05:30  1509.62           NaN            NaN   
2024-09-02 00:00:00+05:30  1464.82           NaN            NaN   
2024-09-09 00:00:00+05:30  1472.62           NaN            NaN   
2024-09-16 00:00:00+05:30  1485.93           NaN            NaN   
2024-09-23 00:00:00+05:30  1526.18           NaN            NaN   
2024-09-30 00:00:00+05:30  1386.53           NaN            NaN   
2024-10-07 00:00:00+05:30  1372.10           NaN            NaN   
2024-10-14 00:00:00+05:30  1359.30           NaN            NaN   
2024-10-21 00:00:00+05:30  1327.85           NaN            NaN   
2024-10-28 00:00:00+05:30  1338.65           NaN            NaN   
2024-11-04 00:00:00+05:30  1283.75           NaN            NaN   
2024-11-11 00:00:00+05:30  1267.60           NaN            NaN   
2024-11-18 00:00:00+05:30

In [20]:
       'from=2025-05-28&'
       'sortBy=popularity&'
       'apiKey=1e8c23be6f7f4f74bdf5d799705f43cd')

response = requests.get(url)

print(json.dumps(response.json(), indent=4))
# Parse and summarize news articles using LLM (following main.py pattern)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import asyncio

# Load the model (Gemini via LangChain) - same as main.py
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

# Define a prompt template for news summarization
news_prompt = PromptTemplate(
    input_variables=["title", "description", "content"],
    template="""
    Summarize this news article in exactly 5 lines:
    
    Title: {title}
    Description: {description}
    Content: {content}
    
    Provide a concise, factual summary focusing on key points and implications.
    """
)

# Create the chain
news_chain = LLMChain(llm=llm, prompt=news_prompt)

news_data = response.json()
articles = news_data.get('articles', [])

print("=== AI-GENERATED NEWS SUMMARY (100 LINES) ===\n")

async def summarize_article(article, index):
    """Summarize a single article using LLM"""
    title = article.get('title', 'No title')
    description = article.get('description', 'No description')
    content = article.get('content', 'No content')
    published_at = article.get('publishedAt', 'No date')
    
    # Clean and format the date
    if published_at != 'No date':
        date_obj = pd.to_datetime(published_at)
        formatted_date = date_obj.strftime('%B %d, %Y')
    else:
        formatted_date = 'Unknown date'
    
    # Generate AI summary using the same pattern as main.py
    try:
        loop = asyncio.get_event_loop()
        ai_summary = await loop.run_in_executor(
            None, 
            lambda: news_chain.run({
                "title": title,
                "description": description,
                "content": content
            })
        )
        
        return {
            "index": index,
            "title": title,
            "date": formatted_date,
            "summary": ai_summary,
            "source": article.get('source', {}).get('name', 'Unknown source')
        }
        
    except Exception as e:
        # Fallback to basic summary if LLM fails
        return {
            "index": index,
            "title": title,
            "date": formatted_date,
            "summary": f"Error with AI summary: {str(e)}. Basic summary: {description[:150]}{'...' if len(description) > 150 else ''}",
            "source": article.get('source', {}).get('name', 'Unknown source')
        }

# Process articles asynchronously (same pattern as main.py)
async def process_all_articles():
    tasks = [summarize_article(article, i+1) for i, article in enumerate(articles[:20])]  # Limit to 20 articles for 100 lines
    results = await asyncio.gather(*tasks)
    
    for result in results:
        print(f"Article {result['index']}: {result['title']}")
        print(f"Published: {result['date']}")
        print(f"AI Summary:")
        print(result['summary'])
        print(f"Source: {result['source']}")
        print("-" * 80)
        print()

# Run the async function
await process_all_articles()

print("=== END OF AI-GENERATED NEWS SUMMARY ===")
print(f"Total articles processed: {len(articles[:20])}")
print(f"Total news sources: {len(set(article.get('source', {}).get('name', 'Unknown') for article in articles[:20]))}")




{
    "status": "ok",
    "totalResults": 486,
    "articles": [
        {
            "source": {
                "id": null,
                "name": "MacRumors"
            },
            "author": "Joe Rossignol",
            "title": "Apple Says These 13 Vehicle Brands Will Soon Offer iPhone Car Keys",
            "description": "In 2020, Apple added a digital car key feature to its Wallet app, allowing users to lock, unlock, and start a compatible vehicle with an iPhone or Apple Watch. The feature is currently offered by select automakers, including Audi, BMW, Hyundai, Kia, Genesis, \u2026",
            "url": "https://www.macrumors.com/2025/06/09/more-automakers-to-offer-iphone-car-keys/",
            "urlToImage": "https://images.macrumors.com/t/GikJ7XlE7MAnyjwKuOBQaF4sR6I=/1944x/article-new/2025/06/iPhone-Car-Key-WWDC-2025.jpg",
            "publishedAt": "2025-06-09T21:38:38Z",
            "content": "In 2020, Apple added a digital car key feature to its Wallet app, allowing u

/home/this/lmk/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_66062/4198777591.py:38: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  news_chain = LLMChain(llm=llm, prompt=news_prompt)
/tmp/ipykernel_66062/4198777591.py:64: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lambda: news_chain.run({


=== AI-GENERATED NEWS SUMMARY (100 LINES) ===



Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing

TypeError: 'NoneType' object is not subscriptable

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 32
}
].


In [1]:
import requests
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

llm = GoogleGenerativeAI(model="gemini-2.5-flash")

api_key = "sk-live-y26ZKLKJ3vgUgQ0AIzA8vVWUMDf1UCWWhhA25QtG"
response = requests.get(
    "https://stock.indianapi.in/recent_announcements",
    params={
      "stock_name": "TATAMOTORS",
    },
    headers={
        "X-Api-Key": api_key
    }
)
print(response.json())
# Extract the response data
announcement_data = response.json()

# Create a prompt for Gemini to analyze the announcements
announcement_prompt = PromptTemplate(
    input_variables=["stock_name", "announcement_data"],
    template="""
Analyze the following recent announcements for {stock_name} and provide insights:

Announcement Data:
{announcement_data}

Please provide:
1. Key highlights from the announcements
2. Potential impact on stock price
3. Important dates or deadlines mentioned
4. Any regulatory or compliance implications
5. Overall sentiment (positive/negative/neutral)

Format your response in a clear, structured manner.
"""
)

# Create LLM chain for announcement analysis
announcement_chain = LLMChain(llm=llm, prompt=announcement_prompt)

# Get AI analysis of announcements
try:
    announcement_analysis = await announcement_chain.arun(
        stock_name="TATAMOTORS",
        announcement_data=json.dumps(announcement_data, indent=2)
    )
    
    print("=== AI ANALYSIS OF RECENT ANNOUNCEMENTS ===")
    print(announcement_analysis)
    print("=" * 50)
    
except Exception as e:
    print(f"Error analyzing announcements: {str(e)}")
    print("Raw announcement data:")
    print(json.dumps(announcement_data, indent=2))


[{'title': 'Closure of Trading Window 23 Jun', 'link': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=ef198888-5cac-46da-b5bb-e36a62e773ca.pdf', 'date': '23 Jun'}, {'title': 'Shareholder Meeting / Postal Ballot-Outcome of AGM 20 Jun', 'link': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=01a085d3-266c-4007-8340-3ee2f477c125.pdf', 'date': '20 Jun'}, {'title': "Shareholder Meeting / Postal Ballot-Scrutinizer''s Report 20 Jun", 'link': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=8edb138d-d45d-4e58-97c6-6557db14617e.pdf', 'date': '20 Jun'}, {'title': "Announcement under Regulation 30 (LODR)-Investor Presentation 16 Jun - Investor presentation to be made on Jaguar Land Rove Automotive PLC''s Investor Day.", 'link': 'https://www.bseindia.com/stockinfo/AnnPdfOpen.aspx?Pname=64f9f5c9-f0e4-4285-8575-0f06404a38ed.pdf', 'date': "16 Jun - Investor presentation to be made on Jaguar Land Rove Automotive PLC''s Investor Day."}, {'title': "Announcement under Regula